# Imports

In [9]:
import numpy as np
import pandas as pd
import os
import scipy

import xlrd
import argparse

import matplotlib.pyplot as plt
%matplotlib inline

import matplotlib.gridspec as gridspec

import seaborn as sns
sns.set(style='whitegrid')

%config InteractiveShell.ast_node_interactivity='all'
%config InlineBackend.figure_format = 'svg'
import warnings; warnings.simplefilter('ignore')

# Data Pre-processing

In [10]:
data_folder = '/home/rdmtinez/Desktop/MScThesis/data/tecan_results_parsed/'
sc2 =  'screen_2.tsv'


In [12]:
filepath = os.path.join(data_folder, sc2)
dfsc2 = pd.read_csv(filepath, sep='\t')

# total samples
dfsc2.shape
dfsc2['Wavelength'].unique()

(16896, 17)

array(['750_nm', 'Ex:440_nm/Em:680_nm'], dtype=object)

In [4]:
# lowercase all columns 
dfsc2.columns = map(str.lower, dfsc2.columns)

# rename columns
rename_cols = {'measurement_blank_average':'blank_measure_mean', 
               'measurement_fixed':'corrected_measure',
               'measurement':'measure','t (h)': 't(h)'}
dfsc2 = dfsc2.rename(rename_cols, axis=1)

# rename day values
day_rename = {'day0':'0', 'day1':'1', 'day2':'2', 'day3':'3',
              'day4':'4', 'day5':'5', 'day6':'6', 'day7':'7'}
dfsc2['day'] = dfsc2['day'].map(day_rename)

#rename and lowercase meas. type values
type_rename = {'Fluorescence Bottom Reading': 'fluorescence', 'Absorbance':'absorbance'}
dfsc2['measurement_type'] = dfsc2['measurement_type'].map(type_rename)

# rename and lowercase wavelength values
wave_rename = {'Ex:440_nm/Em:680_nm':'680_nm', '750_nm':'750_nm'}
dfsc2['wavelength'] = dfsc2['wavelength'].map(wave_rename)

# convert categorical int values to string types
dfsc2['replicate'] = dfsc2['replicate'].map(str)
dfsc2['column'] = dfsc2['column'].map(str)
dfsc2['t(h)'] = dfsc2['t(h)'].map(int)

# create well column to replace row & column 
dfsc2['well'] = dfsc2['row']+dfsc2['column']


# drop columns
dfsc2 =  dfsc2.drop(labels=['sampling_date', 'sampling_time',
                            'filename', 'row', 'column'], axis=1)


# sort values by using the measurement times
dfsc2 = dfsc2.sort_values(by=['sampling_datetime'])
dfsc2 = dfsc2.reset_index(drop=True)

In [5]:
dfsc2[dfsc2['condition']=='sc+c'].head()

,sampling_datetime,day,media,condition,replicate,measurement_type,wavelength,measure,t(h),content,blank_measure_mean,corrected_measure,well
0,2019-02-15 15:57:26,0,tp,sc+c,3,fluorescence,680_nm,23.0,0,Chlamy,0.0,23.0,H12
1,2019-02-15 15:57:26,0,tp,sc+c,3,fluorescence,680_nm,22.0,0,ICL_202C,0.0,22.0,A4
2,2019-02-15 15:57:26,0,tp,sc+c,3,fluorescence,680_nm,14.0,0,ICL_139A,0.0,14.0,B4
3,2019-02-15 15:57:26,0,tp,sc+c,3,fluorescence,680_nm,17.0,0,ICL_208B,0.0,17.0,C4
4,2019-02-15 15:57:26,0,tp,sc+c,3,fluorescence,680_nm,19.0,0,ICL_179A,0.0,19.0,D4



# Pre-Process Indiv Replicates
    
    The experiement was conducted such that the biologists made 12 separate daily measurements
    encompassing different conditions and a variable number of replicates per condition. Here, 
    we first separate the dataframe into absorbance and fluorescence, then we separate them by
    the 3 conditions (sc+c, b+c, b) into the respective day the measurements were taken. 
    Once the daily measurements have been segregated by condition, we then concatenate the
    respective abs and fluorescence measurements ensuring that the measurements correspond to 
    'content' the main key.


## Absorbance DFs

In [65]:
abn = dfsc2[dfsc2['measurement_type']=='absorbance']
abn.shape[0]

9216

#### SC+C

In [73]:
# absorbance condition: sc+c
df_scc = abn[abn['condition'] == 'sc+c']

all_scc_abs_dfs = []
for day in df_scc['day'].unique():
    
    daily_reps_dfs = []
    for rep in df_scc[df_scc['day']==day]['replicate'].unique():
        day,rep
        
        tdf = df_scc[df_scc['day']==day][df_scc['replicate']==rep]

        tdf = tdf[['content', 'well', 't(h)', 'corrected_measure']]
        
        tdf =  tdf.set_index(keys=['well', 'content'])
        
        
        daily_reps_dfs.append(tdf)
        
    
        tdf
    break
    df = pd.concat(daily_reps_dfs, join='outer', axis=0)
    'concat'
    #df
    df = df.groupby(['well', 'content']).mean()[['t(h)','corrected_measure']]
    'groupby'
    #df
    all_scc_abs_dfs.append(df.reset_index())

('0', '3')

,,t(h),corrected_measure
well,content,,
H12,Chlamy,0,0.002421
F12,ICL_60,0,0.004721
G12,ICL_193,0,-0.004779
B4,ICL_139A,0,0.009221
C4,ICL_208B,0,0.001221
D4,ICL_179A,0,0.004521
E4,ICL_202C,0,0.001721
F4,ICL_139A,0,-0.001479
G4,ICL_208B,0,0.002521


('0', '1')

,,t(h),corrected_measure
well,content,,
H6,Chlamy,0,-0.001279
F12,ICL_60,0,0.002621
H3,ICL_170,0,-0.000779
A4,ICL_202C,0,-0.001579
B4,ICL_139A,0,0.005121
C4,ICL_208B,0,0.004121
D4,ICL_179A,0,0.003121
E4,ICL_202C,0,0.000321
F4,ICL_139A,0,0.007421


('0', '2')

,,t(h),corrected_measure
well,content,,
G12,ICL_193,0,0.000821
H12,Chlamy,0,0.000921
E9,ICL_24,0,0.002821
D9,Chlamy,0,0.011121
C9,ICL_172,0,0.004221
B9,ICL_133,0,0.000421
A9,ICL_24,0,0.000721
H8,Chlamy,0,0.004821
G8,ICL_201,0,0.017021


('0', '4')

,,t(h),corrected_measure
well,content,,
A1,ICL_162,0,0.004521
G12,ICL_193,0,0.005621
C4,ICL_208B,0,0.007221
D4,ICL_179A,0,0.003921
E4,ICL_202C,0,0.002121
F4,ICL_139A,0,0.005721
G4,ICL_208B,0,0.002321
H4,ICL_179A,0,0.009921
A5,ICL_131,0,0.005121


#### B+C

In [67]:
# absorbance condition: b+c
df_bc = abn[abn['condition'] == 'b+c']

all_bc_abs_dfs =  []
for day in df_bc['day'].unique():
    
    daily_reps_dfs = []
    for rep in df_bc[df_bc['day']==day]['replicate'].unique():
        #('day','rep'), day, rep
        
        tdf = df_bc[df_bc['day']==day][df_bc['replicate']==rep]
        tdf = tdf[['content', 'well', 't(h)', 'corrected_measure']]
        
        tdf =  tdf.set_index(keys=['well', 'content'])
        
        
        daily_reps_dfs.append(tdf)
    
    
    
    df = pd.concat(daily_reps_dfs, join='outer', axis=0)
    df = df.groupby(['well', 'content']).mean()[['t(h)','corrected_measure']]
    all_bc_abs_dfs.append(df.reset_index())

#### B

In [9]:
# absorbance condition: b
df_b = abn[abn['condition'] == 'b']

all_b_abs_dfs = []
for day in df_b['day'].unique():
    
    daily_reps_dfs = []
    for rep in df_b[df_b['day']==day]['replicate'].unique():
        #('day','rep'), day, rep
        
        tdf = df_b[df_b['day']==day][df_b['replicate']==rep]
        tdf = tdf[['content', 'well', 't(h)', 'corrected_measure']]
        
        tdf =  tdf.set_index(keys=['well', 'content'])
        
        
        daily_reps_dfs.append(tdf)
    
    
    
    df = pd.concat(daily_reps_dfs, join='outer', axis=0)
    df = df.groupby(['well', 'content']).mean()[['t(h)','corrected_measure']]
    all_b_abs_dfs.append(df.reset_index())

## Fluorescence DFs

In [10]:
fls = dfsc2[dfsc2['measurement_type']=='fluorescence']
fls.shape[0]

7680

#### SC+C

In [11]:
# fluorescence condition: sc+c
df_scc = fls[fls['condition'] == 'sc+c']

all_scc_fls_dfs = []
for day in df_scc['day'].unique():
    
    daily_reps_dfs = []
    for rep in df_scc[df_scc['day']==day]['replicate'].unique():
        #('day','rep'), day, rep
        
        tdf = df_scc[df_scc['day']==day][df_scc['replicate']==rep]
        tdf = tdf[['content', 'well', 't(h)', 'corrected_measure']]
        
        tdf =  tdf.set_index(keys=['well', 'content'])
        
        
        daily_reps_dfs.append(tdf)
    
    
    
    df = pd.concat(daily_reps_dfs, join='outer', axis=0)
    df = df.groupby(['well', 'content']).mean()[['t(h)','corrected_measure']]
    all_scc_fls_dfs.append(df.reset_index())

#### B+C

In [12]:
# absorbance condition: b+c
df_bc = fls[fls['condition'] == 'b+c']

all_bc_fls_dfs =  []
for day in df_bc['day'].unique():
    
    daily_reps_dfs = []
    for rep in df_bc[df_bc['day']==day]['replicate'].unique():
        #('day','rep'), day, rep
        
        tdf = df_bc[df_bc['day']==day][df_bc['replicate']==rep]
        tdf = tdf[['content', 'well', 't(h)', 'corrected_measure']]
        
        tdf =  tdf.set_index(keys=['well', 'content'])
        
        
        daily_reps_dfs.append(tdf)
    
    
    
    df = pd.concat(daily_reps_dfs, join='outer', axis=0)
    df = df.groupby(['well', 'content']).mean()[['t(h)','corrected_measure']]
    all_bc_fls_dfs.append(df.reset_index())

#### B

In [68]:
# absorbance condition: b

# NOTE: there are no Fluorescence measurements
# for this condition hence the empty dataframe

df_b = fls[fls['condition'] == 'b']

all_b_fls_dfs = []
for day in df_b['day'].unique():
    
    daily_reps_dfs = []
    for rep in df_b[df_b['day']==day]['replicate'].unique():
        #('day','rep'), day, rep
        
        tdf = df_b[df_b['day']==day][df_b['replicate']==rep]
        tdf = tdf[['content', 'well', 't(h)', 'corrected_measure']]
        
        tdf =  tdf.set_index(keys=['well', 'content'])
        
        
        daily_reps_dfs.append(tdf)
    
    
    
    df = pd.concat(daily_reps_dfs, join='outer', axis=0)
    df
    df = df.groupby(['well', 'content']).mean()[['t(h)','corrected_measure']]
    all_b_fls_dfs.append(df.reset_index())

# Concatenate Respective Abs-Fluo Measurements

#### SC+C

In [15]:
#all_scc_abs_dfs

conc_scc = []
for ab,fl in zip(all_scc_abs_dfs, all_scc_fls_dfs):
    ab = ab.rename({'corrected_measure':'corr_abs'}, axis=1)
    ab = ab.set_index(keys=['well', 'content', 't(h)'])
    
    fl = fl.rename({'corrected_measure':'corr_fls'}, axis=1)
    fl = fl.set_index(keys=['well', 'content', 't(h)'])
    
    df =  pd.concat([ab,fl], join='outer', axis=1).reset_index()
    
    conc_scc.append(df)


#### B+C

In [16]:
conc_bc = []
for ab,fl in zip(all_bc_abs_dfs, all_bc_fls_dfs):
    ab = ab.rename({'corrected_measure':'corr_abs'}, axis=1)
    ab = ab.set_index(keys=['well', 'content', 't(h)'])
    
    fl = fl.rename({'corrected_measure':'corr_fls'}, axis=1)
    fl = fl.set_index(keys=['well', 'content', 't(h)'])
    
    df =  pd.concat([ab,fl], join='outer', axis=1).reset_index()
    
    conc_bc.append(df)

#### B

In [22]:
conc_b = []
for ab in all_b_abs_dfs:

    ab = ab.rename({'corrected_measure':'corr_abs'}, axis=1)
    #ab = ab.set_index(keys=['well', 'content', 't(h)'])

    conc_b.append(ab)

# Integrity Checking

    First we derive the mapping on the 96-well plate. Note that only the 'B'
    condition has BLANK measurements, and that these blank measurements are
    used to correct the values for all the other wells

In [57]:
mapp_scc = pd.DataFrame()
mapp_bc = pd.DataFrame()
mapp_b = pd.DataFrame()

for cc, bb, aa in zip(conc_scc, conc_bc, conc_b):
    for l in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']:
        
        #cc[cc['well'].str.contains(l)]
        c = cc[cc['well'].str.contains(l)]['content'].values
        mapp_scc[l] = c
        
        b = bb[bb['well'].str.contains(l)]['content'].values
        mapp_bc[l] = b
        
        a = aa[aa['well'].str.contains(l)]['content'].values
        mapp_b[l] = a
    
    cc['t(h)'].unique()
    mapp_scc
    mapp_bc
    mapp_b
    break

array([0])

,A,B,C,D,E,F,G,H
0,ICL_162,ICL_115,ICL_144,ICL_186A,ICL_162,ICL_115,ICL_144,ICL_186A
1,ICL_118B,ICL_20BA,ICL_184BA,Chlamy,ICL_118B,ICL_20BA,ICL_184BA,Chlamy
2,ICL_100,ICL_20BB,ICL_184BB,Chlamy,ICL_100,ICL_20BB,ICL_184BB,Chlamy
3,ICL_108,ICL_60,ICL_193,Chlamy,ICL_108,ICL_60,ICL_193,Chlamy
4,ICL_202A,ICL_199,ICL_41,ICL_186B,ICL_202A,ICL_199,ICL_41,ICL_186B
5,ICL_202B,ICL_101,ICL_114B,ICL_170,ICL_202B,ICL_101,ICL_114B,ICL_170
6,ICL_202C,ICL_139A,ICL_208B,ICL_179A,ICL_202C,ICL_139A,ICL_208B,ICL_179A
7,ICL_131,ICL_103,ICL_117,ICL_179B,ICL_131,ICL_103,ICL_117,ICL_179B
8,ICL_116,ICL_160,ICL_132,Chlamy,ICL_116,ICL_160,ICL_132,Chlamy
9,ICL_112,ICL_129A,ICL_48,Chlamy,ICL_112,ICL_129A,ICL_48,Chlamy


,A,B,C,D,E,F,G,H
0,ICL_162,ICL_115,ICL_144,ICL_186A,ICL_162,ICL_115,ICL_144,ICL_186A
1,ICL_118B,ICL_20BA,ICL_184BA,Chlamy,ICL_118B,ICL_20BA,ICL_184BA,Chlamy
2,ICL_100,ICL_20BB,ICL_184BB,Chlamy,ICL_100,ICL_20BB,ICL_184BB,Chlamy
3,ICL_108,ICL_60,ICL_193,Chlamy,ICL_108,ICL_60,ICL_193,Chlamy
4,ICL_202A,ICL_199,ICL_41,ICL_186B,ICL_202A,ICL_199,ICL_41,ICL_186B
5,ICL_202B,ICL_101,ICL_114B,ICL_170,ICL_202B,ICL_101,ICL_114B,ICL_170
6,ICL_202C,ICL_139A,ICL_208B,ICL_179A,ICL_202C,ICL_139A,ICL_208B,ICL_179A
7,ICL_131,ICL_103,ICL_117,ICL_179B,ICL_131,ICL_103,ICL_117,ICL_179B
8,ICL_116,ICL_160,ICL_132,Chlamy,ICL_116,ICL_160,ICL_132,Chlamy
9,ICL_112,ICL_129A,ICL_48,Chlamy,ICL_112,ICL_129A,ICL_48,Chlamy


,A,B,C,D,E,F,G,H
0,ICL_162,ICL_115,ICL_144,ICL_186A,ICL_162,ICL_115,ICL_144,ICL_186A
1,ICL_118B,ICL_20BA,ICL_184BA,BLANK,ICL_118B,ICL_20BA,ICL_184BA,BLANK
2,ICL_100,ICL_20BB,ICL_184BB,BLANK,ICL_100,ICL_20BB,ICL_184BB,BLANK
3,ICL_108,ICL_60,ICL_193,BLANK,ICL_108,ICL_60,ICL_193,BLANK
4,ICL_202A,ICL_199,ICL_41,ICL_186B,ICL_202A,ICL_199,ICL_41,ICL_186B
5,ICL_202B,ICL_101,ICL_114B,ICL_170,ICL_202B,ICL_101,ICL_114B,ICL_170
6,ICL_202C,ICL_139A,ICL_208B,ICL_179A,ICL_202C,ICL_139A,ICL_208B,ICL_179A
7,ICL_131,ICL_103,ICL_117,ICL_179B,ICL_131,ICL_103,ICL_117,ICL_179B
8,ICL_116,ICL_160,ICL_132,BLANK,ICL_116,ICL_160,ICL_132,BLANK
9,ICL_112,ICL_129A,ICL_48,BLANK,ICL_112,ICL_129A,ICL_48,BLANK
